## DISEÑO DE PACKING LINES UTILIZANDO LA TEORÍA DE COLAS

### Importación de librerías necesarias

In [ ]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt

### Datos obtenidos de la línea

In [ ]:
## Llegadas 
ra = 3 # tasa de llegada (pedidos/minuto)
ta = 0.17 # tiempo medio de llegada entre dos pedidos (minutos)

## Proceso
rp = 2.3 #Capacidad de proceso (pedidos/minuto)
tp = 0.33 # Tiempo Efectivo del proceso (minutos)

### Cálculo del tiempo medio de espera (tq) aplicando la ley de Little y aproximacion Allen - Cunneen

In [ ]:
# Definimos las formlás matemáticas aplicando la teoría de colas. 
##  Notaciones y simbología:
"""
ra: tasa de llegadas
rp: tasa de servicio por estación
m : nº de estaciones en paralelo
u : utilización del sistema
CVA: coef. de variación de llegadas
CVP: coef. de variación de proceso
tq: tiempo medio de espera en cola
"""


def tq_allen_cunneen(ra, rp, m, CVA, CVP):
    u = ra / (m * rp)  # utilización total del sistema
    if u >= 1 or u < 0:
        return float("inf")  # Si la cola es 1 se vuelve inestable y se vuelve infinita)
    tp = 1.0 / rp
    exp = math.sqrt(2 * (m + 1)) - 1
    tq = 0.5 * (CVA**2 + CVP**2) * (u**exp) / (m * (1 - u)) * tp
    return tq


# SOLUCION PROPUESTA 1: cola única -> m estaciones (pooling)
def tq_s1(ra, ta, rp, tp, CVA, CVP, m=2):
    """
    En este caso el número de estaciones es 2 (m=2). Cambia3 m si quieres comparar 3, 4, ...
    'ta' y 'tp' no se usan (tp=1/rp), se mantienen para no romper llamadas.
    """
    return tq_allen_cunneen(ra=ra, rp=rp, m=m, CVA=CVA, CVP=CVP)

# SOLUCION PROPUESTA 2: k líneas dedicadas (2 colas, 1 estación por línea)
def tq_s2(ra, ta, rp, tp, CVA, CVP, k=2, reparto=[0.3,0.7]):
    """
    k: número de líneas dedicadas (colas).
    Podemos repartir la carga en las diferentes lineas con 'reparto' (siempre debe sumar 1), 
    p.ej. reparto=[0.7, 0.3]. Para repartos muy desiguales, habrá que ajustar otros parametros para evitar u=1.
    Devolverá el t_q medio esperado (promedio ponderado).
    """
    if reparto is None:
        reparto = [1.0 / k] * k  # reparto uniforme si no se especifica
    tq_prom = 0.0

    for w in reparto: # w es el peso (fracción de carga) para cada línea
        tq_i = tq_allen_cunneen(ra=ra * w, rp=rp, m=1, CVA=CVA, CVP=CVP)
        tq_prom += w * tq_i
    return tq_prom

### Definición de los parámetros de variabilidad CVA y CVP

In [ ]:
# Se considera que los pedidos se lanzan a intervalos regulares (CVA=0)
CVA = 0

# Crear una lista con diferentes valores de CVP y calcular t_q para cada solución (Valores entre 0 y 3)
x, y_1, y_2 = [], [], []
for p in np.linspace(0,3,31): #Valores de 0 a 3 con paso 0.1, ambos incluidos,en total 31 valores)
    CVP = p
    # CVP value
    x.append(CVP)
    # tq calculation
    y_1.append(tq_s1(ra, ta, rp, tp, CVA, CVP))
    y_2.append(tq_s2(ra, ta, rp, tp, CVA, CVP))
    
# Resultados en un DataFrame
df_result = pd.DataFrame({'CVP': x, 'SOLUCION 1':y_1, 'SOLUCION 2':y_2})
df_result['DIFERENCIA (min)'] = df_result['SOLUCION 2'] - df_result['SOLUCION 1']
df_result

### Gráfico de las soluciones propuestas

In [ ]:
# Plot your results
ax = plt.gca()
df_result.plot(figsize=(10,6), x="CVP", y="SOLUCION 1", ax = ax)
df_result.plot(figsize=(10,6), x="CVP", y="SOLUCION 2", ax = ax, color='r')
plt.style.use("bmh")
plt.xlabel('Coeficiente de variacion del proceso (CVp)')
plt.ylabel('Tiempo de cola (min)')
plt.title('Tiempo de cola en Packing para diferentes valores de CVp')
plt.show()